In [1]:
!pip install gensim

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 40.7 MB/s eta 0:00:00


In [2]:
!pip install python-Levenshtein

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.3/153.3 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 11.9 MB/s eta 0:00:00


In [3]:
import gensim
import pandas as pd

Dataset used: http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Cell_Phones_and_Accessories_5.json.gz

In [6]:
df = pd.read_json("/content/Cell_Phones_and_Accessories_5.json",lines=True)
df.head(2)

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A30TL5EWN6DFXT,120401325X,christina,"[0, 0]",They look good and stick good! I just don't li...,4,Looks Good,1400630400,"05 21, 2014"
1,ASY55RVNIL0UD,120401325X,emily l.,"[0, 0]",These stickers work like the review says they ...,5,Really great product.,1389657600,"01 14, 2014"


In [7]:
df.shape

(194439, 9)

In [8]:
df.reviewText

,reviewText
0,They look good and stick good! I just don't li...
1,These stickers work like the review says they ...
2,These are awesome and make my phone look so st...
3,Item arrived in great time and was in perfect ...
4,"awesome! stays on, and looks great. can be use..."
...,...
194434,Works great just like my original one. I reall...
194435,Great product. Great packaging. High quality a...
194436,"This is a great cable, just as good as the mor..."
194437,I really like it becasue it works well with my...


In [9]:
df.reviewText[2]

'These are awesome and make my phone look so stylish! I have only used one so far and have had it on for almost a year! CAN YOU BELIEVE THAT! ONE YEAR!! Great quality!'

In [10]:
gensim.utils.simple_preprocess("These are awesome and make my phone look so stylish! I have only used one so far and have had it on for almost a year! CAN YOU BELIEVE THAT! ONE YEAR!! Great quality!")

['these',
 'are',
 'awesome',
 'and',
 'make',
 'my',
 'phone',
 'look',
 'so',
 'stylish',
 'have',
 'only',
 'used',
 'one',
 'so',
 'far',
 'and',
 'have',
 'had',
 'it',
 'on',
 'for',
 'almost',
 'year',
 'can',
 'you',
 'believe',
 'that',
 'one',
 'year',
 'great',
 'quality']

In [11]:
review_text = df.reviewText.apply(gensim.utils.simple_preprocess)
review_text

,reviewText
0,"[they, look, good, and, stick, good, just, don..."
1,"[these, stickers, work, like, the, review, say..."
2,"[these, are, awesome, and, make, my, phone, lo..."
3,"[item, arrived, in, great, time, and, was, in,..."
4,"[awesome, stays, on, and, looks, great, can, b..."
...,...
194434,"[works, great, just, like, my, original, one, ..."
194435,"[great, product, great, packaging, high, quali..."
194436,"[this, is, great, cable, just, as, good, as, t..."
194437,"[really, like, it, becasue, it, works, well, w..."


In [12]:
model = gensim.models.Word2Vec(
    window=10,
    min_count = 2,
    workers=4
)

In [13]:
model.build_vocab(review_text, progress_per=1000)

In [14]:
model.epochs

5

In [15]:
model.train(review_text, total_examples=model.corpus_count, epochs=model.epochs)

(61505431, 83868975)

In [16]:
model.save("./word2vec-amazon-cell-accessories-reviews-short.model")

In [17]:
model.wv.most_similar("good")

[('decent', 0.8227662444114685),
 ('great', 0.7846043705940247),
 ('nice', 0.705165684223175),
 ('fantastic', 0.6908245086669922),
 ('excellent', 0.6498501300811768),
 ('superb', 0.6276953816413879),
 ('awesome', 0.6058644652366638),
 ('exceptional', 0.596526563167572),
 ('outstanding', 0.5925897359848022),
 ('wonderful', 0.5872716307640076)]

In [18]:
model.wv.most_similar("bad")

[('terrible', 0.6774546504020691),
 ('shabby', 0.6304483413696289),
 ('horrible', 0.616523265838623),
 ('good', 0.5847196578979492),
 ('okay', 0.5528516173362732),
 ('crappy', 0.552022397518158),
 ('awful', 0.5492314696311951),
 ('legit', 0.5446786880493164),
 ('ok', 0.5276309847831726),
 ('sad', 0.526129424571991)]

In [19]:
model.wv.most_similar("product")

[('item', 0.8788362145423889),
 ('products', 0.5218893885612488),
 ('seller', 0.5056489109992981),
 ('value', 0.5039969682693481),
 ('transaction', 0.4936733841896057),
 ('company', 0.49115532636642456),
 ('vendor', 0.4852607250213623),
 ('accessory', 0.4657585620880127),
 ('prodcut', 0.4650135040283203),
 ('merchandise', 0.46070340275764465)]

In [20]:
model.wv.similarity(w1="cheap", w2="inexpensive")

np.float32(0.48939145)

In [21]:
model.wv.similarity(w1="amazon", w2="product")

np.float32(0.3268875)

Exercise: Train a word2vec model on the Sports & Outdoors Reviews Dataset Once you train a model on this, find the words most similar to 'awful' and find similarities between the following word tuples: ('good', 'great'), ('slow','steady')

In [25]:
import json
import pandas as pd

file_path = "/content/Sports_and_Outdoors_5.json"
data = []

with open(file_path, 'r') as f:
    for line_num, line in enumerate(f, 1):
        try:
            data.append(json.loads(line))
        except json.JSONDecodeError as e:
            print(f"Skipping malformed JSON line {line_num}: {line.strip()} - Error: {e}")
            continue



In [27]:
df_exercise = pd.DataFrame(data)
df_exercise.head(2)

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,AIXZKN4ACSKI,1881509818,David Briner,"[0, 0]",This came in on time and I am veru happy with ...,5.0,Woks very good,1390694400,"01 26, 2014"
1,A1L5P841VIO02V,1881509818,Jason A. Kramer,"[1, 1]",I had a factory Glock tool that I was using fo...,5.0,Works as well as the factory tool,1328140800,"02 2, 2012"


In [28]:
df_exercise.shape

(296337, 9)

In [29]:
df_exercise.reviewText

,reviewText
0,This came in on time and I am veru happy with ...
1,I had a factory Glock tool that I was using fo...
2,If you don't have a 3/32 punch or would like t...
3,This works no better than any 3/32 punch you w...
4,I purchased this thinking maybe I need a speci...
...,...
296332,This is a water bottle done right. It is a ver...
296333,If you're looking for an insulated water bottl...
296334,"This Hydracentials Sporty 25 OZ, double insula..."
296335,As usual I received this item free in exchange...


In [30]:
sports_review_text = df_exercise.reviewText.apply(gensim.utils.simple_preprocess)

In [31]:
sports_review_text

,reviewText
0,"[this, came, in, on, time, and, am, veru, happ..."
1,"[had, factory, glock, tool, that, was, using, ..."
2,"[if, you, don, have, punch, or, would, like, t..."
3,"[this, works, no, better, than, any, punch, yo..."
4,"[purchased, this, thinking, maybe, need, speci..."
...,...
296332,"[this, is, water, bottle, done, right, it, is,..."
296333,"[if, you, re, looking, for, an, insulated, wat..."
296334,"[this, hydracentials, sporty, oz, double, insu..."
296335,"[as, usual, received, this, item, free, in, ex..."


In [32]:
exercise_model = gensim.models.Word2Vec(
    window = 10,
    min_count = 2,
    workers = 4
)

In [33]:
exercise_model.build_vocab(sports_review_text, progress_per=1000)

In [34]:
exercise_model.epochs

5

In [35]:
exercise_model.train(sports_review_text, total_examples=exercise_model.corpus_count, epochs=exercise_model.epochs)

(91336123, 121496535)

In [36]:
exercise_model.save("./sports-model.model")

In [37]:
exercise_model.wv.most_similar("water")

[('beverage', 0.7268157601356506),
 ('liquid', 0.7161301374435425),
 ('liquids', 0.7056663036346436),
 ('sediment', 0.6811261773109436),
 ('drinks', 0.6787459850311279),
 ('milk', 0.6730553507804871),
 ('fridge', 0.6575360894203186),
 ('fluids', 0.6526700258255005),
 ('brita', 0.6485832929611206),
 ('streams', 0.6482617855072021)]

In [38]:
exercise_model.wv.most_similar("awful")

[('horrible', 0.7094975113868713),
 ('terrible', 0.6798813343048096),
 ('unpleasant', 0.6137528419494629),
 ('overwhelming', 0.5867319703102112),
 ('oreo', 0.5757378935813904),
 ('isolated', 0.5693212151527405),
 ('overpowering', 0.565376341342926),
 ('ugly', 0.5612331628799438),
 ('impediment', 0.5580644011497498),
 ('enormous', 0.5574111938476562)]

In [39]:
exercise_model.wv.similarity(w1 = "good", w2 = "great")

np.float32(0.7823641)